# VN Dating Simulator Model Evaluation

**Purpose:** Comprehensive evaluation of the VN-trained LLaMA 3.1 model.

**What this notebook evaluates:**
- Dialogue Quality (perplexity, diversity, repetition)
- Character Consistency (4 VN characters: Monika, Sayori, Natsuki, Yuri)
- Affection Responsiveness (0-100 scale)
- Emotion Appropriateness (12 emotion types)
- Character Differentiation (cross-character comparison)
- Multi-turn Conversation Quality

**Model:** LLaMA 3.1 8B fine-tuned on VN (Doki Doki Literature Club) data

---

## 1. Setup and Imports

In [ ]:
# Check environment
import sys
from pathlib import Path

# Add parent to path
if Path.cwd().name == 'VN':
    sys.path.insert(0, str(Path.cwd().parent.parent))
    print("✓ Running from VN directory")
else:
    print(f"⚠️  Current directory: {Path.cwd()}")
    print("Please run from notebooks/VN/ directory")

In [ ]:
# Core imports
import torch
import json
import numpy as np
import pandas as pd
import re
from datetime import datetime
from collections import defaultdict, Counter
from typing import List, Dict, Tuple

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)
from peft import PeftModel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All imports successful")

In [ ]:
# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

---
## 2. Load VN Model

In [ ]:
# VN Model checkpoint path
VN_CHECKPOINT = "../../checkpoints/dating_sim_vn/final"

print(f"Loading VN model from: {VN_CHECKPOINT}")
print("This may take a few minutes...\n")

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(VN_CHECKPOINT)

print(f"✓ Tokenizer loaded: {tokenizer.__class__.__name__}")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"  Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    VN_CHECKPOINT,
    torch_dtype=torch.float16,
    device_map='auto'
)
model.eval()

print("✓ Model loaded successfully")
total_params = sum(p.numel() for p in model.parameters())
print(f"  Total parameters: {total_params:,}")

---
## 3. VN Character Configuration

In [ ]:
# VN Character descriptions (from training data)
VN_CHARACTERS = {
    'Monika': "You are Monika, the Literature Club president. Confident, intelligent, and caring. You're thoughtful and philosophical, ambitious and kind with a mysterious side.",
    'Sayori': "You are Sayori, the Literature Club vice president. Cheerful, optimistic, and caring. You're warm, friendly, and always try to make others happy, though you hide your own struggles.",
    'Natsuki': "You are Natsuki, a Literature Club member. Tsundere, direct, and passionate. You love manga and baking, and while you act tough, you care deeply about your friends.",
    'Yuri': "You are Yuri, a Literature Club member. Shy, intellectual, and passionate about literature. You're thoughtful and eloquent but can be socially anxious and overly self-conscious."
}

# Emotion guidance mappings (from training)
EMOTION_GUIDANCE = {
    'joy': "The user is happy! Match their enthusiasm and share in their joy.",
    'neutral': "Respond naturally based on the conversation context.",
    'anger': "The user appears upset. Stay calm, be understanding, and don't escalate.",
    'surprise': "Respond naturally based on the conversation context.",
    'sadness': "The user seems down. Be supportive and caring.",
    'fear': "The user seems worried. Be reassuring and supportive.",
    'love': "The user is expressing affection. Respond warmly and genuinely.",
    'caring': "The user is showing care. Appreciate their concern and reciprocate.",
    'curiosity': "The user is curious. Be informative and engaging.",
    'confusion': "The user seems confused. Be clear and patient in your explanation.",
    'excitement': "The user is excited! Share in their enthusiasm.",
    'disappointment': "The user seems disappointed. Be understanding and supportive."
}

print(f"✓ Configured {len(VN_CHARACTERS)} VN characters")
print(f"✓ Configured {len(EMOTION_GUIDANCE)} emotion guidance types")

---
## 4. VN Response Generation Function

In [ ]:
def generate_vn_response(
    character: str,
    user_input: str,
    emotion: str = "neutral",
    affection: int = 50,
    max_new_tokens: int = 200,
    temperature: float = 0.6,
    top_p: float = 0.85,
    verbose: bool = False
) -> str:
    """
    Generate VN character response with affection and emotion tracking.
    
    Args:
        character: Character name (Monika, Sayori, Natsuki, Yuri)
        user_input: User's message
        emotion: User's emotional state
        affection: Affection level 0-100
        max_new_tokens: Max tokens to generate
        temperature: Sampling temperature
        top_p: Nucleus sampling threshold
        verbose: Print debug info
    
    Returns:
        Generated response string
    """
    # Get character description
    char_desc = VN_CHARACTERS.get(
        character,
        f"You are {character} from the Literature Club."
    )
    
    # Get emotion guidance
    emotion_guide = EMOTION_GUIDANCE.get(
        emotion,
        "Respond naturally based on the conversation context."
    )
    
    # Build system prompt
    system_content = f"""{char_desc}

Current affection: {affection}/100
User's emotional state: {emotion}

{emotion_guide}"""
    
    # Build messages
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_input}
    ]
    
    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    if verbose:
        print(f"Prompt length: {len(prompt)} chars")
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_length = inputs['input_ids'].shape[1]
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            min_new_tokens=10,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )
    
    # Decode
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # Clean up
    response = re.sub(r'<[^>]+>\s*', '', response)
    response = ' '.join(response.split())
    
    return response

print("✓ VN generation function ready")

---
## 5. Quick Test - Sample Responses

In [ ]:
# Quick test with each character
test_prompt = "How's the Literature Club going?"

print(f"Test prompt: '{test_prompt}'\n")
print("="*80)

for character in VN_CHARACTERS.keys():
    response = generate_vn_response(
        character=character,
        user_input=test_prompt,
        emotion="neutral",
        affection=50
    )
    print(f"{character}: {response}")
    print("-"*80)

---
## 6. Define Test Scenarios

In [ ]:
# Test cases organized by character
# Format: (user_input, emotion, affection)
VN_TEST_CASES = {
    'Monika': [
        ("How's the Literature Club going?", "neutral", 30),
        ("I really enjoyed your poem today!", "joy", 60),
        ("You seem stressed about the festival.", "neutral", 45),
        ("What's your favorite book?", "curiosity", 50),
        ("I'm having a tough day...", "sadness", 55),
        ("You're an amazing club president!", "joy", 70),
        ("Do you ever feel lonely?", "caring", 65),
        ("What do you think about the future?", "neutral", 40),
        ("I love spending time with you.", "love", 80),
        ("Would you like to get coffee sometime?", "neutral", 75),
    ],
    'Sayori': [
        ("You seem happy today!", "joy", 40),
        ("Is everything okay? You seem a bit off...", "neutral", 25),
        ("Your positivity is contagious!", "joy", 60),
        ("What makes you happiest?", "curiosity", 50),
        ("I'm feeling down today.", "sadness", 55),
        ("You're such a good friend.", "caring", 65),
        ("Do you want to hang out after school?", "neutral", 70),
        ("What's your favorite thing about the club?", "neutral", 45),
        ("I really care about you.", "love", 75),
        ("You always know how to cheer me up!", "joy", 80),
    ],
    'Natsuki': [
        ("What are you reading?", "neutral", 20),
        ("Your manga collection is really impressive!", "joy", 50),
        ("Can you teach me about manga?", "curiosity", 35),
        ("I tried your cupcakes - they're amazing!", "joy", 60),
        ("You're not as tough as you pretend to be.", "neutral", 55),
        ("I think you're really talented.", "neutral", 65),
        ("What's your favorite manga series?", "curiosity", 40),
        ("You seem upset about something.", "caring", 45),
        ("I really like spending time with you.", "love", 70),
        ("Your poetry is actually really good!", "joy", 75),
    ],
    'Yuri': [
        ("Tell me about your favorite book.", "neutral", 35),
        ("I'd love to hear you read your poetry.", "joy", 65),
        ("What are your hobbies?", "curiosity", 40),
        ("Would you like to go out for tea?", "neutral", 60),
        ("I love your hair.", "neutral", 20),
        ("You're looking beautiful today.", "neutral", 55),
        ("You seem really knowledgeable about literature.", "neutral", 50),
        ("I'm interested in learning more about horror novels.", "curiosity", 70),
        ("You have such elegant taste.", "joy", 75),
        ("I feel comfortable talking to you.", "caring", 80),
    ]
}

total_tests = sum(len(cases) for cases in VN_TEST_CASES.values())
print(f"✓ Defined {total_tests} test cases across {len(VN_TEST_CASES)} characters")
for char, cases in VN_TEST_CASES.items():
    print(f"  {char}: {len(cases)} tests")

---
## 7. Run Character-Specific Tests

In [ ]:
# Run all test cases and collect responses
from tqdm.notebook import tqdm

all_results = []

print("Running VN character tests...\n")

for character, test_cases in VN_TEST_CASES.items():
    print(f"Testing {character}...")
    
    for user_input, emotion, affection in tqdm(test_cases, desc=character):
        response = generate_vn_response(
            character=character,
            user_input=user_input,
            emotion=emotion,
            affection=affection
        )
        
        all_results.append({
            'character': character,
            'user_input': user_input,
            'emotion': emotion,
            'affection': affection,
            'response': response,
            'response_length': len(response.split())
        })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)

print(f"\n✓ Generated {len(results_df)} responses")
print(f"\nSample results:")
print(results_df.head())

---
## 8. Basic Dialogue Quality Metrics

In [ ]:
# Response length statistics
print("="*80)
print("Response Length Statistics")
print("="*80)

length_stats = results_df['response_length'].describe()
print(f"Mean:   {length_stats['mean']:.1f} words")
print(f"Median: {length_stats['50%']:.1f} words")
print(f"Min:    {length_stats['min']:.0f} words")
print(f"Max:    {length_stats['max']:.0f} words")
print(f"Std:    {length_stats['std']:.1f} words")

In [ ]:
# Calculate distinct-n metrics
def calculate_distinct_n(texts: List[str], n: int) -> float:
    """Calculate distinct-n score for lexical diversity."""
    all_ngrams = []
    
    for text in texts:
        tokens = text.lower().split()
        ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
        all_ngrams.extend(ngrams)
    
    if len(all_ngrams) == 0:
        return 0.0
    
    return len(set(all_ngrams)) / len(all_ngrams)

responses = results_df['response'].tolist()

distinct_1 = calculate_distinct_n(responses, 1)
distinct_2 = calculate_distinct_n(responses, 2)
distinct_3 = calculate_distinct_n(responses, 3)

print("\n" + "="*80)
print("Lexical Diversity (Distinct-N)")
print("="*80)
print(f"Distinct-1 (unique words):    {distinct_1:.4f}")
print(f"Distinct-2 (unique bigrams):  {distinct_2:.4f}")
print(f"Distinct-3 (unique trigrams): {distinct_3:.4f}")

print("\n📊 Interpretation:")
print("  Good: Distinct-1 > 0.3, Distinct-2 > 0.6")
if distinct_1 > 0.3:
    print("  ✓ Good vocabulary diversity")
else:
    print("  ⚠ Low vocabulary diversity")
    
if distinct_2 > 0.6:
    print("  ✓ Good phrase diversity")
else:
    print("  ⚠ Low phrase diversity")

In [ ]:
# Calculate repetition ratio
def calculate_repetition(text: str) -> float:
    """Calculate self-repetition ratio."""
    tokens = text.lower().split()
    if len(tokens) <= 1:
        return 0.0
    
    bigrams = [tuple(tokens[i:i+2]) for i in range(len(tokens)-1)]
    if len(bigrams) == 0:
        return 0.0
    
    repetitions = len(bigrams) - len(set(bigrams))
    return repetitions / len(bigrams)

results_df['repetition'] = results_df['response'].apply(calculate_repetition)
mean_repetition = results_df['repetition'].mean()

print("\n" + "="*80)
print("Self-Repetition Analysis")
print("="*80)
print(f"Mean Repetition Ratio: {mean_repetition:.4f}")

print("\n📊 Interpretation:")
if mean_repetition < 0.05:
    print("  ✓ Excellent - Very little repetition")
elif mean_repetition < 0.1:
    print("  ✓ Good - Acceptable repetition")
elif mean_repetition < 0.2:
    print("  ⚠ Fair - Some repetition present")
else:
    print("  ⚠ Poor - High repetition")

---
## 9. Character-Specific Analysis

In [ ]:
# Per-character statistics
print("="*80)
print("Per-Character Response Quality")
print("="*80)

char_stats = results_df.groupby('character').agg({
    'response_length': ['mean', 'std'],
    'repetition': 'mean'
}).round(2)

char_stats.columns = ['Avg Length', 'Std Length', 'Avg Repetition']
print(char_stats)

# Per-character distinct-n
print("\n" + "="*80)
print("Per-Character Lexical Diversity")
print("="*80)

char_diversity = []
for character in VN_CHARACTERS.keys():
    char_responses = results_df[results_df['character'] == character]['response'].tolist()
    char_diversity.append({
        'Character': character,
        'Distinct-1': calculate_distinct_n(char_responses, 1),
        'Distinct-2': calculate_distinct_n(char_responses, 2),
        'Distinct-3': calculate_distinct_n(char_responses, 3)
    })

diversity_df = pd.DataFrame(char_diversity)
print(diversity_df.round(4))

---
## 10. Affection Responsiveness Test

In [ ]:
# Test same prompt at different affection levels
affection_test_prompt = "Would you like to spend some time together?"
affection_levels = [20, 50, 80]

print("="*80)
print("Affection Responsiveness Test")
print("="*80)
print(f"Test prompt: '{affection_test_prompt}'\n")

affection_results = []

for character in VN_CHARACTERS.keys():
    print(f"\n{character}:")
    print("-"*60)
    
    for affection in affection_levels:
        response = generate_vn_response(
            character=character,
            user_input=affection_test_prompt,
            emotion="neutral",
            affection=affection
        )
        
        print(f"Affection {affection}/100: {response}")
        
        affection_results.append({
            'character': character,
            'affection': affection,
            'response': response
        })

affection_df = pd.DataFrame(affection_results)

---
## 11. Character Differentiation Test

In [ ]:
# Test how differently characters respond to identical prompts
differentiation_prompts = [
    "What do you think about poetry?",
    "What's your ideal weekend?",
    "How do you handle stress?",
    "What makes you happy?",
    "Tell me about yourself."
]

print("="*80)
print("Character Differentiation Test")
print("="*80)
print("Testing if characters give distinctly different responses to identical prompts\n")

differentiation_results = []

for prompt in differentiation_prompts:
    print(f"\nPrompt: '{prompt}'")
    print("-"*60)
    
    responses_for_prompt = []
    
    for character in VN_CHARACTERS.keys():
        response = generate_vn_response(
            character=character,
            user_input=prompt,
            emotion="neutral",
            affection=50
        )
        
        print(f"{character}: {response}")
        
        responses_for_prompt.append(response)
        
        differentiation_results.append({
            'prompt': prompt,
            'character': character,
            'response': response
        })
    
    # Calculate uniqueness
    unique_words = len(set(' '.join(responses_for_prompt).lower().split()))
    total_words = len(' '.join(responses_for_prompt).lower().split())
    uniqueness_ratio = unique_words / total_words if total_words > 0 else 0
    
    print(f"  → Uniqueness ratio: {uniqueness_ratio:.3f}")

differentiation_df = pd.DataFrame(differentiation_results)

---
## 12. Emotion Appropriateness Analysis

In [ ]:
# Analyze emotion distribution in test cases
print("="*80)
print("Emotion Distribution in Tests")
print("="*80)

emotion_dist = results_df['emotion'].value_counts()
print(emotion_dist)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
emotion_dist.plot(kind='bar', ax=ax, color='skyblue')
ax.set_xlabel('Emotion', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Emotion Distribution in Test Cases', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---
## 13. Visualizations

In [ ]:
# Character distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Response length by character
ax = axes[0, 0]
results_df.boxplot(column='response_length', by='character', ax=ax)
ax.set_title('Response Length Distribution by Character')
ax.set_xlabel('Character')
ax.set_ylabel('Words')
plt.sca(ax)
plt.xticks(rotation=0)

# 2. Repetition by character
ax = axes[0, 1]
char_repetition = results_df.groupby('character')['repetition'].mean()
char_repetition.plot(kind='bar', ax=ax, color='coral')
ax.set_title('Mean Repetition by Character')
ax.set_xlabel('Character')
ax.set_ylabel('Repetition Ratio')
ax.axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='Target <0.1')
ax.legend()
plt.xticks(rotation=0)

# 3. Diversity comparison
ax = axes[1, 0]
diversity_df.set_index('Character')[['Distinct-1', 'Distinct-2', 'Distinct-3']].plot(
    kind='bar', ax=ax
)
ax.set_title('Lexical Diversity by Character')
ax.set_xlabel('Character')
ax.set_ylabel('Score')
ax.legend(title='Metric')
plt.xticks(rotation=0)

# 4. Affection distribution in tests
ax = axes[1, 1]
results_df['affection'].hist(bins=20, ax=ax, color='lightgreen', edgecolor='black')
ax.set_title('Affection Level Distribution in Tests')
ax.set_xlabel('Affection (0-100)')
ax.set_ylabel('Frequency')
ax.axvline(x=50, color='red', linestyle='--', alpha=0.5, label='Neutral (50)')
ax.legend()

plt.tight_layout()
plt.show()

---
## 14. Problem Detection

In [ ]:
# Detect potential issues
print("="*80)
print("Problem Detection")
print("="*80)

problems = []

# Check for character name confusion
for idx, row in results_df.iterrows():
    response_lower = row['response'].lower()
    character = row['character']
    
    # Check if response mentions other character names
    for other_char in VN_CHARACTERS.keys():
        if other_char != character and other_char.lower() in response_lower:
            problems.append({
                'type': 'character_confusion',
                'character': character,
                'issue': f"Mentions '{other_char}' in response",
                'response': row['response'][:100] + "..."
            })

# Check for very short responses
short_responses = results_df[results_df['response_length'] < 10]
for idx, row in short_responses.iterrows():
    problems.append({
        'type': 'too_short',
        'character': row['character'],
        'issue': f"Only {row['response_length']} words",
        'response': row['response']
    })

# Check for high repetition
high_repetition = results_df[results_df['repetition'] > 0.2]
for idx, row in high_repetition.iterrows():
    problems.append({
        'type': 'high_repetition',
        'character': row['character'],
        'issue': f"Repetition ratio: {row['repetition']:.3f}",
        'response': row['response'][:100] + "..."
    })

problems_df = pd.DataFrame(problems)

if len(problems_df) > 0:
    print(f"\n⚠️  Found {len(problems_df)} potential issues:\n")
    
    # Group by type
    for problem_type, group in problems_df.groupby('type'):
        print(f"\n{problem_type.upper()} ({len(group)} cases):")
        print("-"*60)
        for idx, row in group.head(3).iterrows():
            print(f"  Character: {row['character']}")
            print(f"  Issue: {row['issue']}")
            print(f"  Response: {row['response']}")
            print()
else:
    print("\n✓ No major issues detected!")

---
## 15. Summary Report

In [ ]:
# Generate comprehensive summary
print("="*80)
print("VN MODEL EVALUATION SUMMARY")
print("="*80)

print(f"\n📊 OVERALL METRICS")
print("-"*60)
print(f"Total test cases:      {len(results_df)}")
print(f"Characters tested:     {len(VN_CHARACTERS)}")
print(f"Avg response length:   {results_df['response_length'].mean():.1f} words")
print(f"Distinct-1 score:      {distinct_1:.4f}")
print(f"Distinct-2 score:      {distinct_2:.4f}")
print(f"Mean repetition:       {mean_repetition:.4f}")

print(f"\n👥 CHARACTER PERFORMANCE")
print("-"*60)
for character in VN_CHARACTERS.keys():
    char_data = results_df[results_df['character'] == character]
    avg_length = char_data['response_length'].mean()
    avg_rep = char_data['repetition'].mean()
    print(f"{character:12s}: Avg {avg_length:4.1f} words, Repetition {avg_rep:.4f}")

print(f"\n⚠️  ISSUES DETECTED")
print("-"*60)
if len(problems_df) > 0:
    issue_counts = problems_df['type'].value_counts()
    for issue_type, count in issue_counts.items():
        print(f"{issue_type:20s}: {count} cases")
else:
    print("No major issues detected ✓")

print(f"\n✅ QUALITY ASSESSMENT")
print("-"*60)

# Overall quality score
quality_score = 0
max_score = 5

if distinct_1 > 0.3:
    quality_score += 1
    print("✓ Good vocabulary diversity")
else:
    print("✗ Low vocabulary diversity")

if distinct_2 > 0.6:
    quality_score += 1
    print("✓ Good phrase diversity")
else:
    print("✗ Low phrase diversity")

if mean_repetition < 0.1:
    quality_score += 1
    print("✓ Low repetition")
else:
    print("✗ High repetition")

if len(problems_df[problems_df['type'] == 'character_confusion']) == 0:
    quality_score += 1
    print("✓ No character confusion detected")
else:
    print("✗ Character confusion present")

if results_df['response_length'].mean() > 15:
    quality_score += 1
    print("✓ Adequate response length")
else:
    print("✗ Responses too short")

print(f"\nOverall Quality Score: {quality_score}/{max_score}")

if quality_score >= 4:
    print("🎉 Excellent model performance!")
elif quality_score >= 3:
    print("👍 Good model performance with room for improvement")
else:
    print("⚠️  Model needs significant improvement")

---
## 16. Save Results

In [ ]:
# Create results directory
results_dir = Path("../../results")
results_dir.mkdir(exist_ok=True)

# Generate timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save detailed results
results_path = results_dir / f"vn_evaluation_{timestamp}.csv"
results_df.to_csv(results_path, index=False)
print(f"✓ Detailed results saved to: {results_path}")

# Save affection test results
affection_path = results_dir / f"vn_affection_test_{timestamp}.csv"
affection_df.to_csv(affection_path, index=False)
print(f"✓ Affection test saved to: {affection_path}")

# Save differentiation test results
diff_path = results_dir / f"vn_differentiation_test_{timestamp}.csv"
differentiation_df.to_csv(diff_path, index=False)
print(f"✓ Differentiation test saved to: {diff_path}")

# Save summary metrics
summary = {
    'timestamp': timestamp,
    'model': VN_CHECKPOINT,
    'total_tests': len(results_df),
    'characters': list(VN_CHARACTERS.keys()),
    'metrics': {
        'avg_response_length': float(results_df['response_length'].mean()),
        'distinct_1': float(distinct_1),
        'distinct_2': float(distinct_2),
        'distinct_3': float(distinct_3),
        'mean_repetition': float(mean_repetition)
    },
    'per_character': char_stats.to_dict(),
    'issues': problems_df.to_dict('records') if len(problems_df) > 0 else [],
    'quality_score': f"{quality_score}/{max_score}"
}

summary_path = results_dir / f"vn_evaluation_summary_{timestamp}.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"✓ Summary saved to: {summary_path}")

---
## 17. Evaluation Complete! 🎉

### Key Findings:
- Review the summary above for overall performance
- Check character-specific metrics for individual performance
- Examine problem detection for areas needing improvement

### Next Steps:
1. **If quality score < 3:** Consider retraining with more data or different hyperparameters
2. **If character confusion detected:** Filter training data to single-character conversations
3. **If repetition high:** Increase repetition penalty in generation
4. **If diversity low:** Check training data diversity, consider data augmentation

### Files Generated:
- `results/vn_evaluation_TIMESTAMP.csv` - Detailed test results
- `results/vn_affection_test_TIMESTAMP.csv` - Affection responsiveness
- `results/vn_differentiation_test_TIMESTAMP.csv` - Character differentiation
- `results/vn_evaluation_summary_TIMESTAMP.json` - Complete summary